In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q twstock

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.1/626.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
import csv
from twstock import Stock
import numpy as np

# 求某年至2024 3月之前的資訊
def get_stock_info_between_years_to_csv(stock_id, start_year=2000):
    # 使用 twstock 模組的 Stock 類來獲取股票資訊
    stock = Stock(stock_id)

    # 初始化一個空列表來存儲資訊
    all_info = []
    # 取從開始年到現今的紀錄
    info = stock.fetch_from(start_year, 1)
    all_info.extend(info)
    # 找到2024年3月之後的第一個索引
    for i, data in enumerate(all_info):
        if data.date.year == 2024 and data.date.month >= 3:
            break
    else:
        i = len(all_info)  # 如果找不到2024年3月之後的資訊，保留全部

    # 刪除2024年3月之後的紀錄
    all_info = all_info[:i]
    # 將資訊寫入 CSV 檔案
    filename = f"/content/drive/MyDrive/LSTM/input/stock_train/{stock_id}_{start_year}_to_202402.csv"
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile)

        # 寫入 CSV 標題行
        csvwriter.writerow(['Open', 'High', 'Low', 'Close', 'Change', 'Volume' , 'Date'])

        # 寫入資訊行
        for data in all_info:
            csvwriter.writerow([data.open, data.high, data.low, data.close, data.change, data.capacity, data.date.strftime('%Y-%m-%d')])

    print(f"已將 {stock_id} 在 {start_year} 年至 2024 年 2 月的資訊儲存為 {filename}")

# 使用範例
# stock_id = '3711'  # 台積電的股票代碼
# start_year = 1985  # 開始年份
# get_stock_info_between_years_to_csv(stock_id, start_year)
get_stock_info_between_years_to_csv('2330', 2000)

已將 2330 在 2000 年至 2024 年 2 月的資訊儲存為 /content/drive/MyDrive/LSTM/input/stock_train/2330_2000_to_202402.csv


In [ ]:
"""
以下是從上市時間就開始取資料的方法，但我認為這樣的資訊會有問題，所以就全部預設2010
"""
# get_stock_info_between_years_to_csv('2330', 1994)
# get_stock_info_between_years_to_csv('2317', 1991)
# get_stock_info_between_years_to_csv('2454', 2001)
# get_stock_info_between_years_to_csv('2382', 1999)
# get_stock_info_between_years_to_csv('2412', 2000)
# get_stock_info_between_years_to_csv('2881', 2001)
# get_stock_info_between_years_to_csv('2308', 1988)
# get_stock_info_between_years_to_csv('2882', 2001)
# get_stock_info_between_years_to_csv('2891', 2002)
# get_stock_info_between_years_to_csv('3711', 2018)

In [ ]:
import csv
from twstock import Stock
stock = Stock('2330')
info = stock.fetch_from(2024, 6)
print(info)

[Data(date=datetime.datetime(2024, 6, 3, 0, 0), capacity=29629706, turnover=25053781742, open=839.0, high=853.0, low=837.0, close=846.0, change=25.0, transaction=40694), Data(date=datetime.datetime(2024, 6, 4, 0, 0), capacity=31031104, turnover=26107162214, open=844.0, high=851.0, low=837.0, close=839.0, change=-7.0, transaction=52392), Data(date=datetime.datetime(2024, 6, 5, 0, 0), capacity=37531781, turnover=31774150536, open=841.0, high=857.0, low=835.0, close=854.0, change=15.0, transaction=41869)]


In [ ]:
import twstock
import datetime
import pickle
import numpy as np
import torch
import torch.nn as nn

######## 路徑修改 ########
model_path = "/content/drive/MyDrive/LSTM/model/LSTM_class_2024_06_10_14_00_56_3"
scaler_path = '/content/drive/MyDrive/LSTM/input/stock_train/scaler.pkl'
######## 路徑修改 ########


#############################
### 模型載入並且開啟評估模式
# 檢查CUDA是否可用
USE_CUDA = torch.cuda.is_available()
# 設定GPU可用則使用cuda:0，不可用則用CPU
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

# 定義相同的LSTM
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim, dropout):
        # 調用父類 nn.Module 的構造函數
        super(LSTM, self).__init__()
        # 設定 LSTM 模型的隱藏層維度
        self.hidden_dim = hidden_dim
        # 設定 LSTM 模型的層數
        self.num_layers = num_layers
        # 創建 LSTM 層，指定輸入維度、隱藏層維度、層數和批次優先的設置
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        # 創建全連接層，將隱藏層的輸出映射到輸出維度
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # 初始化 LSTM 的隱藏狀態和細胞狀態，設置為全零，並移動到指定設備
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)
        # 將輸入數據移動到指定設備，並傳遞給 LSTM 層，得到輸出和新的隱藏狀態、細胞狀態
        out, (hn, cn) = self.lstm(x.to(device), (h0.detach(), c0.detach()))
        # 將 LSTM 的輸出傳遞給全連接層，僅使用最後一個時間步的輸出
        out = self.fc(out[:, -1, :])
        # 將最終輸出移動到指定設備並返回
        return out.to(device)
# 設定超參數
# 獲取輸入數據的維度
input_dim = 5
# 設定 LSTM 模型的隱藏層維度
hidden_dim = 200
# 設定 LSTM 模型的層數
num_layers = 2
# 設定模型的輸出維度
output_dim = 3
# 設定LSTM堆疊的dropout層
dropout = 0.001


model = LSTM(input_dim, hidden_dim, num_layers, output_dim, dropout).to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()  # 開啟評估模式
### 模型載入並且開啟評估模式
#############################
def date_com(data_date, today):
    if isinstance(data_date, str):
        data_date = datetime.datetime.strptime(data_date, '%Y-%m-%d').date()
    if isinstance(today, str):
        today = datetime.datetime.strptime(today, '%Y-%m-%d').date()

    return data_date <= today

def get_recent_nine_days_data(stock_id='2330'):
    # 獲取當前日期
    today = datetime.datetime.today().date()
    # 取得股票資料
    stock = twstock.Stock(stock_id)
    all_data = stock.fetch_31()  # 取最新31天的資料
    if(len(all_data)<10):
      return None
    # 篩選最近9天的資料
    recent_nine_days_data = []
    for data in all_data:
        # 確保所有日期比較都是同一類型
        #print(data.date.strftime('%Y-%m-%d'))
        data_date = data.date.strftime('%Y-%m-%d')
        if date_com(data_date, today):
            recent_nine_days_data.append([
                data.open, # 0
                data.high, # 1
                data.low, # 2
                data.close, # 3
                data.change, # 4
                data.capacity, # 5
                data_date, # 6
            ])

    # 按日期排序
    # recent_nine_days_data.sort(key=lambda x: x['Date'])
    recent_nine_days_data.sort(key=lambda x: x[6])
    return recent_nine_days_data
def stock_change_predicted(stock_id):
  stock_data = get_recent_nine_days_data(stock_id=stock_id)[-10:]
  if (stock_data == None):
    return "請求的股票資訊過少 或 twstock套件發生異常"
  else:
    # 將日期從資料中移除
    data_values = [entry[0:6] for entry in stock_data]
    #print(data_values[0])
    # 加载scaler对象
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)
    #scaler = MinMaxScaler()
    def get_features(data):
      result = []
      for sublist in data:
          # 提取索引为0到3以及5的元素
          extracted_values = sublist[0:4] + [sublist[5]]
          result.append(extracted_values)
      return result
    features = get_features(data_values)
    scaled_stock_data = scaler.fit_transform(features)
    #print(scaled_stock_data)
    new_data_tensor = torch.tensor([scaled_stock_data], dtype=torch.float32).to(device)

    outputs = model(new_data_tensor)

    _, predicted = torch.max(outputs, 1)
    predicted = predicted[0].detach().cpu().numpy()

    if(predicted==1):
      return "長" # "今天收盤比昨天高"
    elif(predicted==0):
      return "同" # "今天收盤比昨天一樣價"
    else:
      return "跌" # "今天收盤可能比昨天低"


######## 預測執行 ########
stock_id = input("查詢股票id:")
print(stock_change_predicted(stock_id))
######## 預測執行 ########


查詢股票id:2317
跌
